In [131]:
import os
import pickle
import requests
import subprocess
import statistics
from collections import defaultdict

import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [142]:
user_seq = Seq(input("Provide a DNA sequence:"))

Provide a DNA sequence: CACTTAATGCATGCGATCGGCCATTAT GTAATAGATCATAATCCTTCTGCCAAAGTGGTTTATCTGTCTTCTGAGAAATTTACAAACGAATTCATCA ACTCTATCCGAGATAATAAAGCCGTCGACTTCCGCAATCGCTATCGAAATGTTGATGTGCTTTTGATAGA TGATATTCAATTTTTAGCGGGGAAAGAACAAACCCAGGAAGAATTTTTCCATACATTTAACACATTACAC GAAGAAAGCAAACAAATCGTCATTTCAAGTGACCGGCCGCCAAAGGAAATTCCGACACTTGAAGACAGAT TGCGCTCACGTTTTGAATGGGGACTTATTACAGATATCACACCGCCTGATCTAGAAACGAGAATTGCAAT TTTAAGAAAAAAGGCCAAAGCAGAGGGCCTCGATATTCCGAACGAGGTTATGCTTTACATCGCGAATCAA ATCGACAGCAATATTCGGGAACTCGAAGGAGCATTAATCAGAGTTGTCGCTTATTCATCTTTAATTAATA AAGATATTAATGCTGATCTGGCCGCTGAGGCGTTGAAAGATATTATTCCTTCCTCAAAACCGAAAGTCAT TACGATAAAAGAAATTCAGAGGGTAGTAGGCCAGCAATTTAATATTAAACTCGAGGATTTCAAAGCAAAA AAACGGACAAAGTCAGTAGCTTTTCCGCGTCAAATCGCCATGTACTTATCAAGGGAAATGACTGATTCCT CTCTTCCTAAAATCGGTGAAGAGTTTGGAGGACGTGATCATACGACCGTTATTCATGCGCATGAAAAAAT TTCAAAACTGCTGGCAGATGATGAACAGCTTCAGCAGCATGTAAAAGAAATTAAAGAACAGCTTAAATAG CAGGACCGGGGATCAATCGGGGAAAGTGTGAATAACTTTTCGGAAGTCATACACAGTCTGTCCACATGTG GATAGGCTGTGTTTCCTGTCTTTTT

In [143]:
# read in the model and list of canonical 5-mers
clf = pickle.load(open("kmer_SVM.sav", 'rb'))
kmer_list = pickle.load(open("kmer_list.pkl", 'rb'))

In [144]:
def is_valid_sequence(seq):
    ''' Used to remove any kmers with N's in them '''
    return set(seq).issubset({"A", "T", "C", "G"})

kmer_dict = {}
for j in range(len(user_seq)):
    kmer = user_seq[j:j+5].upper()
    if kmer.reverse_complement() in kmer_list:
        kmer = kmer.reverse_complement()
    if len(kmer) == 5:
        if is_valid_sequence(kmer) is True:
            if kmer in kmer_dict:
                kmer_dict[str(kmer)] += 1
            else:
                kmer_dict[str(kmer)] = 1
                
for kmer in kmer_list:
    if kmer not in kmer_dict:
        kmer_dict[kmer] = 0

/home/zpimentel/miniconda3/envs/VirTect/lib/python3.6/site-packages/Bio/Seq.py:155: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


In [145]:
kmer_df = pd.DataFrame.from_dict(kmer_dict, orient='index')
kmer_df = kmer_df.reindex(sorted(kmer_list))
kmer_df = kmer_df.div(kmer_df.sum(axis=0),axis=1)
contig_kmers = kmer_df[0].values.tolist()
contig_kmers = np.array(contig_kmers).reshape(1, -1)

In [146]:
predictions = clf.predict(contig_kmers)

if predictions == 1:
    print("This appears to be eukaryotic.")
elif predictions == 0:
    print("This appears to be bacterial.")
else:
    print("Something went wrong =(")

This appears to be bacterial.
